딥러닝을 이용한 자연어 처리 입문 03. 문서유사도

#### 1) 코사인 유사도(Cosine Similarity)

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [2]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

In [3]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


#### 유사도를 이용한 추천 시스템 구현하기

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [5]:
# data download: https://www.kaggle.com/rounakbanik/the-movies-dataset

In [6]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [9]:
# 20,000건 만 훈련데이터 축소 

data = data.head(20000)

In [10]:
# null 값 제거 
data['overview'].isnull().sum() 

135

In [12]:
# overview에서 Null 값을 가진 경우에는 Null 값을 제거
data['overview'] = data['overview'].fillna('') 
# 빈 값(empty value)으로 대체하여 Null 값을 제거

In [13]:
data['overview'].isnull().sum()

0

In [19]:
# tf-idf를 수행

tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


In [23]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
tfidf.vocabulary_

{'led': 24361,
 'woody': 46617,
 'andy': 2051,
 'toys': 43131,
 'live': 24957,
 'happily': 18727,
 'room': 36185,
 'birthday': 4736,
 'brings': 5775,
 'buzz': 6290,
 'lightyear': 24759,
 'scene': 37166,
 'afraid': 1254,
 'losing': 25229,
 'place': 32291,
 'heart': 19047,
 'plots': 32424,
 'circumstances': 7974,
 'separate': 37778,
 'owner': 30780,
 'duo': 12920,
 'eventually': 14393,
 'learns': 24325,
 'aside': 2811,
 'differences': 11645,
 'siblings': 38509,
 'judy': 22536,
 'peter': 31851,
 'discover': 11888,
 'enchanted': 13728,
 'board': 5054,
 'game': 16798,
 'opens': 30240,
 'door': 12442,
 'magical': 25725,
 'world': 46664,
 'unwittingly': 44575,
 'invite': 21663,
 'alan': 1495,
 'adult': 1135,
 'trapped': 43304,
 'inside': 21286,
 '26': 430,
 'years': 46965,
 'living': 24973,
 'hope': 19854,
 'freedom': 16355,
 'finish': 15562,
 'proves': 33496,
 'risky': 35885,
 'running': 36455,
 'giant': 17265,
 'rhinoceroses': 35691,
 'evil': 14422,
 'monkeys': 28035,
 'terrifying': 42245,


In [15]:
# 코사인 유사도를 사용(문서의 유사도 구하기 위함)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [26]:
cosine_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

In [27]:
cosine_sim.shape

(20000, 20000)

In [28]:
pd.Series(data.index, index=data['title'])

title
Toy Story                                                                       0
Jumanji                                                                         1
Grumpier Old Men                                                                2
Waiting to Exhale                                                               3
Father of the Bride Part II                                                     4
                                                                            ...  
Rebellion                                                                   19995
Versailles                                                                  19996
Two in the Wave                                                             19997
Lotte Reiniger: Homage to the Inventor of the Silhouette Film               19998
RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'    19999
Length: 20000, dtype: int64

In [29]:
# 영화의 타이틀과 인덱스를 가진 테이블 생성
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
# 똑같은 타이틀이 있으면 제거 drop_duplicates()
print(indices.head())
# 테이블의 용도: 영화의 타이틀을 입력하면 인덱스를 리턴하기 위함

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [30]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [40]:
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores

[(0, 0.0),
 (1, 0.0),
 (2, 0.025004916790732457),
 (3, 0.0),
 (4, 1.0),
 (5, 0.0),
 (6, 0.03297982155878723),
 (7, 0.0),
 (8, 0.03275127428366323),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.01617538079378207),
 (17, 0.0),
 (18, 0.022556258634846782),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.010453269553250327),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.012483050325811314),
 (34, 0.0),
 (35, 0.0),
 (36, 0.008551449198000282),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.012535288895276211),
 (42, 0.005066317867374504),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.026180146785198196),
 (48, 0.0),
 (49, 0.0),
 (50, 0.030452575775893928),
 (51, 0.008835118316997882),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.0),
 (56, 0.017932910857610264),
 (57, 0.0),
 (58, 0.018584923398927675),
 (59, 0.0),
 (60, 0.016195519400571387),
 (61, 0.0

In [41]:
# 유사도에 따라 영화들을 정렬합니다.
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores

[(4, 1.0),
 (6793, 0.3066645162650513),
 (6571, 0.28316406751428747),
 (6306, 0.28098375143123),
 (19801, 0.27428589995325126),
 (5005, 0.25002558636746086),
 (13611, 0.24972568693001884),
 (7097, 0.23264970441562285),
 (926, 0.22125958103902596),
 (13420, 0.21070467647993246),
 (5571, 0.20133216314749064),
 (13928, 0.19495304005974154),
 (6813, 0.1854562521664336),
 (10260, 0.18464872567140178),
 (5749, 0.1814926882450688),
 (1516, 0.1814328492891251),
 (4112, 0.17797827120546403),
 (10381, 0.17759557909499227),
 (13383, 0.17270231161174054),
 (9174, 0.17017504843202785),
 (15155, 0.16982360199830038),
 (18310, 0.168777143249199),
 (14489, 0.1616845958249809),
 (19090, 0.15989420886266686),
 (14931, 0.15880012153144313),
 (11248, 0.15693012287862962),
 (555, 0.1562763070981479),
 (2823, 0.1557353839461149),
 (5819, 0.15435733119100803),
 (15837, 0.1526092592572152),
 (7540, 0.1457246918788849),
 (6715, 0.14499991830853598),
 (14939, 0.14151086165260637),
 (16064, 0.1398745072292049),


In [44]:
# 가장 유사한 10개의 영화를 받아옵니다.
sim_scores = sim_scores[1:11]
# [0]: 자기 자신(영화타이틀) 제외하고 상위 10개 추출
sim_scores

[(6571, 0.28316406751428747),
 (6306, 0.28098375143123),
 (19801, 0.27428589995325126),
 (5005, 0.25002558636746086),
 (13611, 0.24972568693001884),
 (7097, 0.23264970441562285),
 (926, 0.22125958103902596),
 (13420, 0.21070467647993246),
 (5571, 0.20133216314749064)]

In [45]:
# 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
movie_indices = [i[0] for i in sim_scores]
movie_indices

[6571, 6306, 19801, 5005, 13611, 7097, 926, 13420, 5571]

In [46]:
data['title'].iloc[movie_indices]

6571                     Kuffs
6306           North to Alaska
19801                  Babbitt
5005                   Wendigo
13611      The Magic of Méliès
7097        The Out of Towners
926      It's a Wonderful Life
13420             Funny People
5571            All Night Long
Name: title, dtype: object

In [47]:
# 선택한 영화에 대해서 코사인 유사도를 이용, 
# 가장 overview가 유사한 10개의 영화를 찾아내는 함수

def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [48]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

#### 여러가지 유사도 기법 

- 유클리디언 유사도

In [49]:
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


- 해석
 - 유클리드 거리의 값이 가장 작다는 것: 문서 간의 거리가 가장 가깝다는 것을 의미
 - 즉 doc1과 doc2가 가장 유사함    

- 자카드 유사도(Jaccard similarity)
 -  합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것이 자카드 유사도(jaccard similarity)의 아이디어

In [50]:
# 다음과 같은 두 개의 문서가 있습니다.
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화를 수행합니다.
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [51]:
# 문서1과 문서2의 합집합 구하기 

union = set(tokenized_doc1).union(set(tokenized_doc2))
# set()사용, 중복 제거 
print(union)

{'banana', 'likey', 'you', 'card', 'passport', 'coupon', 'apple', 'love', 'everyone', 'holder', 'like', 'watch'}


In [52]:
len(union) # 문서1과 문서2의 합집합의 단어의 총 개수는 12개

12

In [53]:
# 문서1과 문서2의 교집합 구하기
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'banana', 'apple'}


In [54]:
len(intersection)

2

In [55]:
print(len(intersection)/len(union)) # 2를 12로 나눔.
# 자카드 유사도: 16.6% 
# 두 문서의 총 단어 집합에서 두 문서에서 공통적으로 등장한 단어의 비율

0.16666666666666666
